<a href="https://colab.research.google.com/github/BinduGGowda/Cost-comparsion/blob/main/Cost-Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:

import pandas as pd
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter



In [21]:
def generate_cost_comparison_report(files, sheet_name="Data"):
    if len(files) < 2 or len(files) > 3:
        raise ValueError("You must provide 2 files for Monthly report or 3 files for Quarterly report.")

    # Determine report type and output file name
    if len(files) == 2:
        report_type = "Monthly Cost Comparison Report"
        output_file = "Monthly_Cost_Comparison_Report.xlsx"
    else:  # len(files) == 3
        report_type = "Quarterly Cost Report"
        output_file = "Quarterly_Cost_Report.xlsx"

    # Step 1: Read and aggregate all files
    agg_list = []
    file_names = []
    for file in files:
        df = pd.read_excel(file, sheet_name=sheet_name, engine="openpyxl")
        agg = df.groupby(["ResourceType", "Squad"])["CostUSD"].sum().reset_index()
        agg_list.append(agg)
        file_names.append(os.path.splitext(os.path.basename(file))[0])  # ✅ Only file name

    # ✅ Rename columns before merging to avoid duplicates
    for i in range(len(agg_list)):
        agg_list[i].rename(columns={"CostUSD": f"Cost {file_names[i]} in USD"}, inplace=True)

    # ✅ Merge all renamed DataFrames
    comparison = agg_list[0]
    for i in range(1, len(agg_list)):
        comparison = pd.merge(comparison, agg_list[i], on=["ResourceType", "Squad"], how="outer").fillna(0)

    # Step 3: Logic for Monthly vs Quarterly
    if len(files) == 2:  # Monthly comparison
        for i in range(1, len(file_names)):
            diff_col = f"Difference ({file_names[i]} vs {file_names[0]}) in USD"
            pct_col = f"Difference Percentage ({file_names[i]} vs {file_names[0]}) %"
            comparison[diff_col] = comparison[f"Cost {file_names[i]} in USD"] - comparison[f"Cost {file_names[0]} in USD"]
            comparison[pct_col] = comparison.apply(
                lambda row: 0 if row[f"Cost {file_names[0]} in USD"] == 0 else abs(round(((row[f"Cost {file_names[i]} in USD"] - row[f"Cost {file_names[0]} in USD"]) / row[f"Cost {file_names[0]} in USD"] * 100), 2)),
                axis=1
            )
    else:  # Quarterly report
        cost_cols = [f"Cost {name} in USD" for name in file_names]
        comparison["Total Cost in USD"] = comparison[cost_cols].sum(axis=1)
        # ✅ Keep only Squad, ResourceType, 3 cost columns, and Total Cost
        comparison = comparison[["Squad", "ResourceType"] + cost_cols + ["Total Cost in USD"]]

    # Sort by Squad and ResourceType
    comparison.sort_values(by=["Squad", "ResourceType"], inplace=True)

    # Summary by file
    total_costs = []
    for i, file in enumerate(files):
        df = pd.read_excel(file, sheet_name=sheet_name, engine="openpyxl")
        total_costs.append(df["CostUSD"].sum())
    summary = pd.DataFrame({"File": file_names, "TotalCostUSD": total_costs})

    # Squad-level summary
    squad_summary = None
    for i, file in enumerate(files):
        df = pd.read_excel(file, sheet_name=sheet_name, engine="openpyxl")
        squad_cost = df.groupby("Squad")["CostUSD"].sum().reset_index().rename(columns={"CostUSD": f"Cost {file_names[i]} in USD"} )
        squad_summary = squad_cost if squad_summary is None else pd.merge(squad_summary, squad_cost, on="Squad", how="outer").fillna(0)

    # Create Excel workbook
    wb = Workbook()

    # Detailed Comparison sheet
    ws1 = wb.active
    ws1.title = report_type
    for r in dataframe_to_rows(comparison, index=False, header=True):
        ws1.append(r)

    # Conditional formatting for Diff columns (only for monthly)
    if len(files) == 2:
        diff_cols = [col for col in comparison.columns if col.startswith("Difference (")]
        for col_name in diff_cols:
            col_index = comparison.columns.get_loc(col_name) + 1
            for row in range(2, ws1.max_row + 1):
                cell = ws1.cell(row=row, column=col_index)
                if cell.value > 0:
                    cell.fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")  # Red for positive diff
                elif cell.value < 0:
                    cell.fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Green for negative diff

    # Summary sheet
    ws2 = wb.create_sheet(title="Summary")
    ws2.append([report_type])
    for r in dataframe_to_rows(summary, index=False, header=True):
        ws2.append(r)

    # Chart 1: Total Cost by File
    chart1 = BarChart()
    chart1.title = f"{report_type} - Total Cost by File"
    chart1.x_axis.title = "File"
    chart1.y_axis.title = "Total Cost (USD)"
    data1 = Reference(ws2, min_col=2, min_row=2, max_row=ws2.max_row)
    cats1 = Reference(ws2, min_col=1, min_row=3, max_row=ws2.max_row)
    chart1.add_data(data1, titles_from_data=True)
    chart1.set_categories(cats1)
    chart1.type = "col"
    chart1.style = 10
    ws2.add_chart(chart1, "E2")

    # Add Squad Summary table
    ws2.append([])
    ws2.append(["Squad Summary"])
    for r in dataframe_to_rows(squad_summary, index=False, header=True):
        ws2.append(r)

    # Chart 2: Squad Cost Comparison
    start_row = ws2.max_row - len(squad_summary) + 1
    cats2 = Reference(ws2, min_col=1, min_row=start_row + 1, max_row=ws2.max_row)
    chart2 = BarChart()
    chart2.title = f"{report_type} - Squad Cost Comparison"
    chart2.x_axis.title = "Squad"
    chart2.y_axis.title = "Cost (USD)"
    chart2.type = "col"
    chart2.style = 10
    chart2.grouping = "clustered"

    for i in range(2, 2 + len(file_names)):
        series = Reference(ws2, min_col=i, min_row=start_row, max_row=ws2.max_row)
        chart2.add_data(series, titles_from_data=True)

    chart2.set_categories(cats2)
    ws2.add_chart(chart2, "E20")

    # Add Top Differences sheet ONLY for 2 files
    if len(files) == 2:
        top10 = comparison.sort_values(by=f"Difference ({file_names[1]} vs {file_names[0]}) in USD", ascending=False).head(10)
        ws3 = wb.create_sheet(title="Top Differences")
        for r in dataframe_to_rows(top10, index=False, header=True):
            ws3.append(r)

    # ✅ Auto-adjust column widths for ALL sheets
    for ws in wb.worksheets:
        for col in ws.columns:
            max_length = max(len(str(cell.value)) for cell in col)
            ws.column_dimensions[get_column_letter(col[0].column)].width = max_length + 2

    # Save workbook
    wb.save(output_file)
    print(f"✅ {report_type} generated successfully: {output_file}")




In [25]:

# ✅ Interactive user input for file paths
file_list = []
print("Enter 2 or 3 Excel file paths. Type 'done' when finished:")

while True:
    path = input(f"Enter file path {len(file_list)+1}: ").strip()
    if path.lower() == "done":
        break
    if path:
        file_list.append(path)
    if len(file_list) == 3:  # ✅ Stop asking after 3 files
        print("✅ Maximum files reached (3). Proceeding with Quarterly report...")
        break
    if len(file_list) >= 2 and len(file_list) < 3:
        more = input("Do you want to add another file? (yes/no): ").strip().lower()
        if more == "no":
            break

# ✅ Validate input
if len(file_list) < 2 or len(file_list) > 3:
    raise ValueError("You must provide 2 files for Monthly report or 3 files for Quarterly report.")

# Generate report
generate_cost_comparison_report(files=file_list, sheet_name="Data")

Enter 2 or 3 Excel file paths. Type 'done' when finished:
Enter file path 1: /content/LSEGSaaSPPR_CostAnalysis_Sep2025.xlsx
Enter file path 2: /content/LSEGSaaSPRD_CostAnalysis_Sep2025 1.xlsx
Do you want to add another file? (yes/no): no
✅ Monthly Cost Comparison Report generated successfully: Monthly_Cost_Comparison_Report.xlsx
